In [1]:
import pandas as pd
import geopandas as gpd
import xarray as xr
import os
from pyproj import Transformer
from scipy.spatial import cKDTree
import numpy as np
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

In [2]:
print(os.getcwd())

/home/franc/Desktop/ProjIntegrador


In [3]:
apa2017 = pd.read_excel('apa/PM10_2017-01-01_2017-12-31.xlsx', header=0)
apa2018 = pd.read_excel('apa/PM10_2018-01-01_2018-12-31.xlsx', header=0)

In [4]:
print(f"Apa 2017 dimensões: {apa2017.shape}")
print(apa2017.head())

Apa 2017 dimensões: (8760, 61)
        PM10 (µg/m3)\t  Alverca  Anta-Espinho  Arcos  Aveiro  \
0  2017-01-01 00:00:00     45.5           NaN   48.8    86.0   
1  2017-01-01 01:00:00     32.2           NaN   48.0    80.0   
2  2017-01-01 02:00:00     31.4           NaN   46.8    75.0   
3  2017-01-01 03:00:00     26.6           NaN   43.3    73.0   
4  2017-01-01 04:00:00     26.1           NaN   37.3    71.0   

   Avenida da Liberdade  Avintes  Burgães-Santo Tirso  \
0                  69.2      NaN                  NaN   
1                  69.7      NaN                  NaN   
2                  55.4      NaN                  NaN   
3                  53.0      NaN                  NaN   
4                  42.7      NaN                  NaN   

   Cascais - Escola da Cidadela  Cerro  ...  Reboleira  Santa Cruz de Benfica  \
0                          55.0   23.1  ...       75.5                   70.9   
1                          52.2   20.5  ...       66.9                   48.7  

In [5]:
print(f"Apa 2018 dimensões: {apa2018.shape}")
print(apa2018.head())

Apa 2018 dimensões: (8760, 61)
        PM10 (µg/m3)\t  Alverca  Anta-Espinho  Arcos  Aveiro  \
0  2018-01-01 00:00:00     54.4           NaN   55.5    85.0   
1  2018-01-01 01:00:00     41.6           NaN   54.8    96.0   
2  2018-01-01 02:00:00     40.0          35.0   53.6    96.0   
3  2018-01-01 03:00:00     21.9          30.0   48.7    79.0   
4  2018-01-01 04:00:00     20.0          33.0   39.8    79.0   

   Avenida da Liberdade  Avintes  Burgães-Santo Tirso  \
0                  64.9      NaN                 22.0   
1                  68.1      NaN                 43.0   
2                  79.0      NaN                  NaN   
3                  83.4      NaN                  NaN   
4                  79.4     43.0                  NaN   

   Cascais - Escola da Cidadela  Cerro  ...  Reboleira  Santa Cruz de Benfica  \
0                          61.9   23.7  ...       50.0                    NaN   
1                          63.3   27.2  ...       70.2                    NaN  

In [6]:
print(type(apa2017.iloc[0,0]))
apa2017["PM10 (µg/m3)\t"] = pd.to_datetime(apa2017["PM10 (µg/m3)	"], format='%Y-%m-%d %H:%M:%S')
print(type(apa2017.iloc[0,0]))
apa2018["PM10 (µg/m3)\t"] = pd.to_datetime(apa2018["PM10 (µg/m3)	"], format='%Y-%m-%d %H:%M:%S')

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [7]:
"""IMPORTANTE: EM 2017 A COLUNA DE ERVEDEIRA ESTÁ FORMATADA COMO "object" E NÃO "float64" POR ALGUMA RAZÃO"""

print("2017 numeric columns:", len(apa2017.select_dtypes(include='number').columns.tolist()))
print("2018 numeric columns:", len(apa2018.select_dtypes(include='number').columns.tolist()))
print("2017 localidades:", apa2017.shape[1]-1)
print("2018 localidades:", apa2018.shape[1]-1)
non_numeric_2017 = apa2017.columns.difference(apa2017.select_dtypes(include='number').columns)
print("Non-numeric 2017 columns:", non_numeric_2017)
print(apa2017["Ervedeira"].dtype)

2017 numeric columns: 59
2018 numeric columns: 60
2017 localidades: 60
2018 localidades: 60
Non-numeric 2017 columns: Index(['Ervedeira', 'PM10 (µg/m3)\t'], dtype='object')
object


In [8]:
apa2017["Ervedeira"] = pd.to_numeric(apa2017["Ervedeira"].astype(str).str.strip(), errors='coerce')
print(apa2017["Ervedeira"].dtype)  # Should output: float64


float64


In [9]:
#outubro de 2017 (14-18)
mask = (apa2017["PM10 (µg/m3)\t"] >= pd.to_datetime('2017-10-14')) & (apa2017["PM10 (µg/m3)\t"] < pd.to_datetime('2017-10-19')) #devolve um dicionario de falso ou correto para a condição
Napa2017 = apa2017.loc[mask]
Napa2017 = Napa2017.copy() #só para não haver mutabilidade visto que o dataframe é uma fatia de outro
print(Napa2017)
Napa2017.to_excel("apa/apa2017_14_18outubro.xlsx", index=False)

          PM10 (µg/m3)\t  Alverca  Anta-Espinho  Arcos  Aveiro  \
6864 2017-10-14 00:00:00     47.2           0.0   50.8    34.0   
6865 2017-10-14 01:00:00     42.2           5.0   48.4    37.0   
6866 2017-10-14 02:00:00     43.3          40.0   48.2    33.0   
6867 2017-10-14 03:00:00     45.5          38.0   46.3    29.0   
6868 2017-10-14 04:00:00     46.6          24.0   46.1    28.0   
...                  ...      ...           ...    ...     ...   
6979 2017-10-18 19:00:00      8.9           2.0    8.9    23.0   
6980 2017-10-18 20:00:00      8.3          19.0    8.6    25.0   
6981 2017-10-18 21:00:00      6.7          22.0    5.9    26.0   
6982 2017-10-18 22:00:00      6.9          42.0    5.6    25.0   
6983 2017-10-18 23:00:00      7.8          38.0   12.0    24.0   

      Avenida da Liberdade  Avintes  Burgães-Santo Tirso  \
6864                  82.1     46.0                  1.0   
6865                  69.8     40.0                  0.0   
6866                  68.2 

In [10]:
#junho de 2018 (16-22)
mask = (apa2018["PM10 (µg/m3)\t"] >= pd.to_datetime('2018-06-16')) & (apa2018["PM10 (µg/m3)\t"] < pd.to_datetime('2018-06-23'))
Napa2018 = apa2018.loc[mask]
Napa2018 = Napa2018.copy() #só para não haver mutabilidade visto que o dataframe é uma fatia de outro
print(Napa2018)
Napa2018.to_excel("apa/apa2018_16_23junho.xlsx", index=False)

          PM10 (µg/m3)\t  Alverca  Anta-Espinho  Arcos  Aveiro  \
3984 2018-06-16 00:00:00     18.9          25.0   16.9    20.0   
3985 2018-06-16 01:00:00     17.8           4.0   17.1    19.0   
3986 2018-06-16 02:00:00     10.0           6.0   17.1    19.0   
3987 2018-06-16 03:00:00     10.3          28.0   17.4    17.0   
3988 2018-06-16 04:00:00     12.2          28.0   16.5    18.0   
...                  ...      ...           ...    ...     ...   
4147 2018-06-22 19:00:00      NaN           0.0   24.9    17.0   
4148 2018-06-22 20:00:00      NaN           0.0   25.3    18.0   
4149 2018-06-22 21:00:00      NaN           0.0   31.4    19.0   
4150 2018-06-22 22:00:00      NaN           1.0   32.0    19.0   
4151 2018-06-22 23:00:00      NaN          18.0   34.9    19.0   

      Avenida da Liberdade  Avintes  Burgães-Santo Tirso  \
3984                  29.8     22.0                  0.0   
3985                  26.0     25.0                  0.0   
3986                  25.0 

In [11]:
#a média do dia só é calculada se houver pelo menos 19 valores válidos (≈80%) c.c. = NaN

Napa2017["Data"] = Napa2017["PM10 (µg/m3)\t"].dt.date
Napa2017.to_excel("apa/teste2017.xlsx", index=False)

In [12]:
colunas_numericas = Napa2017.select_dtypes(include='number').columns #se usarmos .mean() no dataframe inteiro dá erro porque a primeira e ultima coluna sao datas e não floats
media2017 = Napa2017.groupby("Data")[colunas_numericas].mean()
print(media2017)

              Alverca  Anta-Espinho      Arcos     Aveiro  \
Data                                                        
2017-10-14  54.300000     42.208333  54.062500  44.791667   
2017-10-15  59.066667     63.333333  49.929167  73.458333   
2017-10-16  64.258333     54.500000  64.600000  93.000000   
2017-10-17  28.808333     17.541667  31.158333  45.000000   
2017-10-18  11.125000      9.458333  10.304167  16.083333   

            Avenida da Liberdade    Avintes  Burgães-Santo Tirso  \
Data                                                               
2017-10-14             61.962500  31.500000             4.222222   
2017-10-15             57.408333  32.000000             1.545455   
2017-10-16             67.820833        NaN            30.600000   
2017-10-17             23.591667  22.571429            23.812500   
2017-10-18             13.220833   9.454545             6.565217   

            Cascais - Escola da Cidadela      Cerro   Chamusca  ...  \
Data                    

In [13]:
obs_diarias = Napa2017.groupby("Data").count()
obs_diarias.to_excel("apa/obs17.xlsx", index=False)
obs_diarias=obs_diarias.iloc[:, 1:]
print(obs_diarias)
mask = obs_diarias >= 5

            Alverca  Anta-Espinho  Arcos  Aveiro  Avenida da Liberdade  \
Data                                                                     
2017-10-14       24            24     24      24                    24   
2017-10-15       24            24     24      24                    24   
2017-10-16       24            24     24      24                    24   
2017-10-17       24            24     24      24                    24   
2017-10-18       24            24     24      24                    24   

            Avintes  Burgães-Santo Tirso  Cascais - Escola da Cidadela  Cerro  \
Data                                                                            
2017-10-14       12                   18                            24     24   
2017-10-15        5                   11                            16     24   
2017-10-16        0                   10                             0     18   
2017-10-17       14                   16                             0     1

In [14]:
print(mask)

            Alverca  Anta-Espinho  Arcos  Aveiro  Avenida da Liberdade  \
Data                                                                     
2017-10-14     True          True   True    True                  True   
2017-10-15     True          True   True    True                  True   
2017-10-16     True          True   True    True                  True   
2017-10-17     True          True   True    True                  True   
2017-10-18     True          True   True    True                  True   

            Avintes  Burgães-Santo Tirso  Cascais - Escola da Cidadela  Cerro  \
Data                                                                            
2017-10-14     True                 True                          True   True   
2017-10-15     True                 True                          True   True   
2017-10-16    False                 True                         False   True   
2017-10-17     True                 True                         False   Tru

In [15]:
Fmedia2017 = media2017[mask]
print(Fmedia2017)

              Alverca  Anta-Espinho      Arcos     Aveiro  \
Data                                                        
2017-10-14  54.300000     42.208333  54.062500  44.791667   
2017-10-15  59.066667     63.333333  49.929167  73.458333   
2017-10-16  64.258333     54.500000  64.600000  93.000000   
2017-10-17  28.808333     17.541667  31.158333  45.000000   
2017-10-18  11.125000      9.458333  10.304167  16.083333   

            Avenida da Liberdade    Avintes  Burgães-Santo Tirso  \
Data                                                               
2017-10-14             61.962500  31.500000             4.222222   
2017-10-15             57.408333  32.000000             1.545455   
2017-10-16             67.820833        NaN            30.600000   
2017-10-17             23.591667  22.571429            23.812500   
2017-10-18             13.220833   9.454545             6.565217   

            Cascais - Escola da Cidadela      Cerro   Chamusca  ...  \
Data                    

In [16]:
#Repetir para 2018

Napa2018["Data"] = Napa2018["PM10 (µg/m3)\t"].dt.date
Napa2018.to_excel("apa/teste2018.xlsx", index=False)
colunas_numericas = Napa2018.select_dtypes(include='number').columns #se usarmos .mean() no dataframe inteiro dá erro porque a primeira e ultima coluna sao datas e não floats
media2018 = Napa2018.groupby("Data")[colunas_numericas].mean()
obs_diarias = Napa2018.groupby("Data").count()
obs_diarias.to_excel("apa/obs18.xlsx", index=False)
obs_diarias=obs_diarias.iloc[:, 1:]
mask = obs_diarias >= 5
Fmedia2018 = media2018[mask]

In [17]:
Fmedia2017 = Fmedia2017.replace("", pd.NA)
Fmedia2017["proporção"] = Fmedia2017.notna().sum(axis=1) / (Fmedia2017.shape[1])
print(Fmedia2017)
Fmedia2017.to_excel("apa/F5media2017.xlsx")

Fmedia2018 = Fmedia2018.replace("", pd.NA)
Fmedia2018["proporção"] = (Fmedia2018.notna().sum(axis=1) / (Fmedia2018.shape[1]))
print(Fmedia2018)
Fmedia2018.to_excel("apa/F5media2018.xlsx")

              Alverca  Anta-Espinho      Arcos     Aveiro  \
Data                                                        
2017-10-14  54.300000     42.208333  54.062500  44.791667   
2017-10-15  59.066667     63.333333  49.929167  73.458333   
2017-10-16  64.258333     54.500000  64.600000  93.000000   
2017-10-17  28.808333     17.541667  31.158333  45.000000   
2017-10-18  11.125000      9.458333  10.304167  16.083333   

            Avenida da Liberdade    Avintes  Burgães-Santo Tirso  \
Data                                                               
2017-10-14             61.962500  31.500000             4.222222   
2017-10-15             57.408333  32.000000             1.545455   
2017-10-16             67.820833        NaN            30.600000   
2017-10-17             23.591667  22.571429            23.812500   
2017-10-18             13.220833   9.454545             6.565217   

            Cascais - Escola da Cidadela      Cerro   Chamusca  ...  \
Data                    

In [18]:
gdf = gpd.read_file("estacoes/qualar_estacao_inspire_vw.shp")
print(gdf.columns.tolist())
print(f"Dimensão da info geofŕafica das estações: {gdf.shape}\n")
print(gdf.head())

['codigo_est', 'nome_estac', 'data_inici', 'data_fim', 'latitude', 'longitude', 'rede_codig', 'rede_nome', 'rede_aqd_o', 'geometry']
Dimensão da info geofŕafica das estações: (65, 10)

    codigo_est                      nome_estac  data_inici    data_fim  \
0  STA-PT01053                         Avintes  12/07/2010  0000/00/00   
1  STA-PT01021             Custóias-Matosinhos  01/09/1998  0000/00/00   
2  STA-PT01023               Ermesinde-Valongo  01/10/1998  0000/00/00   
3  STA-PT01024             D.Manuel II-Vermoim  01/11/1999  0000/00/00   
4  STA-PT01028  Francisco Sá Carneiro-Campanha  19/09/2000  0000/00/00   

    latitude  longitude   rede_codig                         rede_nome  \
0  41.098187  -8.554351  NET-PT00015  Rede de Qualidade do Ar do Norte   
1  41.199805  -8.644844  NET-PT00015  Rede de Qualidade do Ar do Norte   
2  41.206605  -8.552600  NET-PT00015  Rede de Qualidade do Ar do Norte   
3  41.235563  -8.618715  NET-PT00015  Rede de Qualidade do Ar do Norte   


In [19]:
cams2017 = xr.open_dataset("cams/PM10_daymean_2017.nc")
print(cams2017)

<xarray.Dataset> Size: 4MB
Dimensions:    (lon: 40, lat: 66, time: 365, bnds: 2)
Coordinates:
  * lon        (lon) float64 320B -10.0 -9.9 -9.8 -9.7 ... -6.4 -6.3 -6.2 -6.1
  * lat        (lat) float64 528B 36.5 36.6 36.7 36.8 ... 42.7 42.8 42.9 43.0
  * time       (time) datetime64[ns] 3kB 2017-01-01T11:30:00 ... 2017-12-31T1...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 6kB ...
    pm10       (time, lat, lon) float32 4MB ...
Attributes:
    CDI:          Climate Data Interface version ?? (http://mpimet.mpg.de/cdi)
    Conventions:  CF-1.7
    history:      Fri Apr 11 12:39:22 2025: cdo daymean PM10_2017.nc PM10_day...
    Title:        CAMS European air quality interim reanalysis
    Provider:     COPERNICUS European air quality service
    Production:   COPERNICUS Atmosphere Monitoring Service
    CDO:          Climate Data Operators version 1.7.0 (http://mpimet.mpg.de/...


In [20]:
print(cams2017.data_vars)
print("---------------------------------------\n")
print(cams2017.coords)
print("---------------------------------------\n")
print(cams2017["pm10"])
print("---------------------------------------")

Data variables:
    time_bnds  (time, bnds) datetime64[ns] 6kB ...
    pm10       (time, lat, lon) float32 4MB ...
---------------------------------------

Coordinates:
  * lon      (lon) float64 320B -10.0 -9.9 -9.8 -9.7 ... -6.4 -6.3 -6.2 -6.1
  * lat      (lat) float64 528B 36.5 36.6 36.7 36.8 36.9 ... 42.7 42.8 42.9 43.0
  * time     (time) datetime64[ns] 3kB 2017-01-01T11:30:00 ... 2017-12-31T11:...
---------------------------------------

<xarray.DataArray 'pm10' (time: 365, lat: 66, lon: 40)> Size: 4MB
[963600 values with dtype=float32]
Coordinates:
  * lon      (lon) float64 320B -10.0 -9.9 -9.8 -9.7 ... -6.4 -6.3 -6.2 -6.1
  * lat      (lat) float64 528B 36.5 36.6 36.7 36.8 36.9 ... 42.7 42.8 42.9 43.0
  * time     (time) datetime64[ns] 3kB 2017-01-01T11:30:00 ... 2017-12-31T11:...
Attributes:
    standard_name:  mass_concentration_of_pm10_ambient_aerosol_in_air
    long_name:      mass concentration of particulate matter with d < 10 µm
    units:          µg/m3
--------------

![1 de janeiro de 2017](imagens/1_janeiro_2017.png)
![15 de outubro de 2017](imagens/15_outubro_2017.png)
![19 de junho de 2018](imagens/19_junho_2018.png)

In [21]:
pm10 = cams2017["pm10"]
cams2017df = pm10.to_dataframe().reset_index()
print(cams2017df.columns.tolist())
print(f"Dimensão dos dados das camaras: {cams2017df.shape}\n")
print(cams2017df.head())
excerto2017 = cams2017df.iloc[:len(cams2017df)//10] #para ver o que se passava é mais fácil ver somente um décimo do dataset visto que é enorme
excerto2017.to_excel("cams/pm10cams2017.xlsx", index=False)


['time', 'lat', 'lon', 'pm10']
Dimensão dos dados das camaras: (963600, 4)

                 time   lat   lon       pm10
0 2017-01-01 11:30:00  36.5 -10.0  18.154917
1 2017-01-01 11:30:00  36.5  -9.9  17.517908
2 2017-01-01 11:30:00  36.5  -9.8  16.446743
3 2017-01-01 11:30:00  36.5  -9.7  16.456955
4 2017-01-01 11:30:00  36.5  -9.6  16.954844


1) Atribuir coordenadas às localidades no dataframe com os valores médios limpos
2) É preciso projeção de coordenadas ou os dados das cams e apa estão ampas no mesmo tipo???? 
    caso não: arredondar visto que nas netCDFs a latitude e longitude estão arredondadas ---> pode levar a problemas depois na comparação
3) Juntar dados num unico dataset (2 no nosso caso, 1 para cada): 
    ordem cronologica
    pm10 apa e pm10 cams
    localidade com as respetivas coordenadas

In [22]:
transformer = Transformer.from_crs("EPSG:4326", "EPSG:3763", always_xy=True)
gdf.to_excel("apa/gdf1.xlsx", index=False)
x, y = transformer.transform(gdf['longitude'].values, gdf['latitude'].values)

gdf["3763_long"] = x
gdf["3763_lat"] = y
print(gdf.head())
gdf.to_excel("apa/gdf2.xlsx", index=False)

    codigo_est                      nome_estac  data_inici    data_fim  \
0  STA-PT01053                         Avintes  12/07/2010  0000/00/00   
1  STA-PT01021             Custóias-Matosinhos  01/09/1998  0000/00/00   
2  STA-PT01023               Ermesinde-Valongo  01/10/1998  0000/00/00   
3  STA-PT01024             D.Manuel II-Vermoim  01/11/1999  0000/00/00   
4  STA-PT01028  Francisco Sá Carneiro-Campanha  19/09/2000  0000/00/00   

    latitude  longitude   rede_codig                         rede_nome  \
0  41.098187  -8.554351  NET-PT00015  Rede de Qualidade do Ar do Norte   
1  41.199805  -8.644844  NET-PT00015  Rede de Qualidade do Ar do Norte   
2  41.206605  -8.552600  NET-PT00015  Rede de Qualidade do Ar do Norte   
3  41.235563  -8.618715  NET-PT00015  Rede de Qualidade do Ar do Norte   
4  41.162791  -8.590847  NET-PT00015  Rede de Qualidade do Ar do Norte   

  rede_aqd_o                   geometry     3763_long       3763_lat  
0   regional  POINT (-8.55435 41.09819)

In [23]:
x1, y1 = transformer.transform(cams2017df['lon'].values, cams2017df['lat'].values)

cams2017df.loc[:, "3763_long"] = x1
cams2017df.loc[:, "3763_lat"] = y1

print(cams2017df.head())


                 time   lat   lon       pm10      3763_long       3763_lat
0 2017-01-01 11:30:00  36.5 -10.0  18.154917 -167265.612117 -350049.532044
1 2017-01-01 11:30:00  36.5  -9.9  17.517908 -158305.172793 -350218.603496
2 2017-01-01 11:30:00  36.5  -9.8  16.446743 -149344.875659 -350378.361097
3 2017-01-01 11:30:00  36.5  -9.7  16.456955 -140384.712685 -350528.806258
4 2017-01-01 11:30:00  36.5  -9.6  16.954844 -131424.675835 -350669.940311


In [24]:
cams2018 = xr.open_dataset("cams/PM10_daymean_2018.nc")
pm10 = cams2018["pm10"]
cams2018df = pm10.to_dataframe().reset_index()
print(cams2018df.columns.tolist())
print(f"Dimensão dos dados das camaras: {cams2018df.shape}\n")
print(cams2018df.head())

['time', 'lat', 'lon', 'pm10']
Dimensão dos dados das camaras: (963600, 4)

                 time   lat   lon       pm10
0 2018-01-01 11:30:00  36.5 -10.0  22.185546
1 2018-01-01 11:30:00  36.5  -9.9  22.135416
2 2018-01-01 11:30:00  36.5  -9.8  22.132486
3 2018-01-01 11:30:00  36.5  -9.7  22.175129
4 2018-01-01 11:30:00  36.5  -9.6  22.148437


In [25]:
x1, y1 = transformer.transform(cams2018df['lon'].values, cams2018df['lat'].values)

cams2018df.loc[:, "3763_long"] = x1
cams2018df.loc[:, "3763_lat"] = y1

print(cams2018df.head())

                 time   lat   lon       pm10      3763_long       3763_lat
0 2018-01-01 11:30:00  36.5 -10.0  22.185546 -167265.612117 -350049.532044
1 2018-01-01 11:30:00  36.5  -9.9  22.135416 -158305.172793 -350218.603496
2 2018-01-01 11:30:00  36.5  -9.8  22.132486 -149344.875659 -350378.361097
3 2018-01-01 11:30:00  36.5  -9.7  22.175129 -140384.712685 -350528.806258
4 2018-01-01 11:30:00  36.5  -9.6  22.148437 -131424.675835 -350669.940311


In [26]:
print(type(cams2017df.iloc[0,0]))
cams2017df["time"] = pd.to_datetime(cams2017df["time"], format='%Y-%m-%d %H:%M:%S')
print(type(cams2017df.iloc[0,0]))
cams2018df["time"] = pd.to_datetime(cams2018df["time"], format='%Y-%m-%d %H:%M:%S')

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [27]:
#outubro de 2017 (14-18)
mask = (cams2017df["time"] >= pd.to_datetime('2017-10-14')) & (cams2017df["time"] < pd.to_datetime('2017-10-19')) #devolve um dicionario de falso ou correto para a condição
camaras2017 = cams2017df.loc[mask]
camaras2017 = camaras2017.copy() #só para não haver mutabilidade visto que o dataframe é uma fatia de outro
print(camaras2017)
camaras2017.to_excel("cams/cams2017_14_18outubro.xlsx", index=False)

                      time   lat   lon       pm10      3763_long  \
755040 2017-10-14 11:30:00  36.5 -10.0  42.453941 -167265.612117   
755041 2017-10-14 11:30:00  36.5  -9.9  42.392410 -158305.172793   
755042 2017-10-14 11:30:00  36.5  -9.8  42.162560 -149344.875659   
755043 2017-10-14 11:30:00  36.5  -9.7  41.917164 -140384.712685   
755044 2017-10-14 11:30:00  36.5  -9.6  42.237843 -131424.675835   
...                    ...   ...   ...        ...            ...   
768235 2017-10-18 11:30:00  43.0  -6.5  16.272829  133166.524410   
768236 2017-10-18 11:30:00  43.0  -6.4  13.677632  141320.872736   
768237 2017-10-18 11:30:00  43.0  -6.3  12.745512  149475.251732   
768238 2017-10-18 11:30:00  43.0  -6.2  12.116454  157629.663144   
768239 2017-10-18 11:30:00  43.0  -6.1  12.044427  165784.108715   

             3763_lat  
755040 -350049.532044  
755041 -350218.603496  
755042 -350378.361097  
755043 -350528.806258  
755044 -350669.940311  
...               ...  
768235  371319.

In [28]:
#junho de 2018 (16-22)
mask = (cams2018df["time"] >= pd.to_datetime('2018-06-16')) & (cams2018df["time"] < pd.to_datetime('2018-06-23')) #devolve um dicionario de falso ou correto para a condição
camaras2018 = cams2018df.loc[mask]
camaras2018 = camaras2018.copy() #só para não haver mutabilidade visto que o dataframe é uma fatia de outro
print(camaras2018)
camaras2018.to_excel("cams/cams2018_16_22junho.xlsx", index=False)

                      time   lat   lon       pm10      3763_long  \
438240 2018-06-16 11:30:00  36.5 -10.0  27.978189 -167265.612117   
438241 2018-06-16 11:30:00  36.5  -9.9  27.848306 -158305.172793   
438242 2018-06-16 11:30:00  36.5  -9.8  27.857421 -149344.875659   
438243 2018-06-16 11:30:00  36.5  -9.7  27.400064 -140384.712685   
438244 2018-06-16 11:30:00  36.5  -9.6  27.009765 -131424.675835   
...                    ...   ...   ...        ...            ...   
456715 2018-06-22 11:30:00  43.0  -6.5  11.020507  133166.524410   
456716 2018-06-22 11:30:00  43.0  -6.4  10.670572  141320.872736   
456717 2018-06-22 11:30:00  43.0  -6.3  10.260986  149475.251732   
456718 2018-06-22 11:30:00  43.0  -6.2  10.341471  157629.663144   
456719 2018-06-22 11:30:00  43.0  -6.1  10.629801  165784.108715   

             3763_lat  
438240 -350049.532044  
438241 -350218.603496  
438242 -350378.361097  
438243 -350528.806258  
438244 -350669.940311  
...               ...  
456715  371319.

In [29]:
gdf_min = gdf[['nome_estac', '3763_long', '3763_lat']]
print(gdf_min.head())

                       nome_estac     3763_long       3763_lat
0                         Avintes -35388.726933  158867.677319
1             Custóias-Matosinhos -42924.770151  170193.796239
2               Ermesinde-Valongo -35183.611562  170907.573844
3             D.Manuel II-Vermoim -40710.864377  174152.466336
4  Francisco Sá Carneiro-Campanha -38417.071842  166057.858413


In [30]:
print(camaras2017.head())
cam2017_estac =  camaras2017.merge(gdf_min, on=['3763_lat', '3763_long'], how='left')
print(cam2017_estac.head())

                      time   lat   lon       pm10      3763_long  \
755040 2017-10-14 11:30:00  36.5 -10.0  42.453941 -167265.612117   
755041 2017-10-14 11:30:00  36.5  -9.9  42.392410 -158305.172793   
755042 2017-10-14 11:30:00  36.5  -9.8  42.162560 -149344.875659   
755043 2017-10-14 11:30:00  36.5  -9.7  41.917164 -140384.712685   
755044 2017-10-14 11:30:00  36.5  -9.6  42.237843 -131424.675835   

             3763_lat  
755040 -350049.532044  
755041 -350218.603496  
755042 -350378.361097  
755043 -350528.806258  
755044 -350669.940311  
                 time   lat   lon       pm10      3763_long       3763_lat  \
0 2017-10-14 11:30:00  36.5 -10.0  42.453941 -167265.612117 -350049.532044   
1 2017-10-14 11:30:00  36.5  -9.9  42.392410 -158305.172793 -350218.603496   
2 2017-10-14 11:30:00  36.5  -9.8  42.162560 -149344.875659 -350378.361097   
3 2017-10-14 11:30:00  36.5  -9.7  41.917164 -140384.712685 -350528.806258   
4 2017-10-14 11:30:00  36.5  -9.6  42.237843 -131424.6758

In [31]:
print(cam2017_estac["nome_estac"].unique())

"""NÃO DEU"""

[nan]


'NÃO DEU'

In [32]:
gdf_min = gdf[['nome_estac', 'longitude', 'latitude']]
gdf_min = gdf_min.rename(columns={
    'latitude': 'lat',
    'longitude': 'lon'
})
cam2017_estac =  camaras2017.merge(gdf_min, on=['lat', 'lon'], how='left')
print(cam2017_estac.head())
print(cam2017_estac["nome_estac"].unique())

"""NÃO DEU"""


                 time   lat   lon       pm10      3763_long       3763_lat  \
0 2017-10-14 11:30:00  36.5 -10.0  42.453941 -167265.612117 -350049.532044   
1 2017-10-14 11:30:00  36.5  -9.9  42.392410 -158305.172793 -350218.603496   
2 2017-10-14 11:30:00  36.5  -9.8  42.162560 -149344.875659 -350378.361097   
3 2017-10-14 11:30:00  36.5  -9.7  41.917164 -140384.712685 -350528.806258   
4 2017-10-14 11:30:00  36.5  -9.6  42.237843 -131424.675835 -350669.940311   

  nome_estac  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  
[nan]


'NÃO DEU'

In [33]:
print(camaras2017[['3763_lat', '3763_long']].drop_duplicates())
print("nº de coordenadas unicas:", len(camaras2017[['3763_lat', '3763_long']].drop_duplicates()))
print("entries no df inteiro:", camaras2017.shape[0])
print(5 * 2640)

             3763_lat      3763_long
755040 -350049.532044 -167265.612117
755041 -350218.603496 -158305.172793
755042 -350378.361097 -149344.875659
755043 -350528.806258 -140384.712685
755044 -350669.940311 -131424.675835
...               ...            ...
757675  371319.110964  133166.524410
757676  371482.524421  141320.872736
757677  371655.653689  149475.251732
757678  371838.499941  157629.663144
757679  372031.064418  165784.108715

[2640 rows x 2 columns]
nº de coordenadas unicas: 2640
entries no df inteiro: 13200
13200


In [34]:
print(camaras2018[['3763_lat', '3763_long']].drop_duplicates())
print("nº de coordenadas unicas:", len(camaras2018[['3763_lat', '3763_long']].drop_duplicates()))
print("entries no df inteiro:", camaras2018.shape[0])
print(7 * 2640)

             3763_lat      3763_long
438240 -350049.532044 -167265.612117
438241 -350218.603496 -158305.172793
438242 -350378.361097 -149344.875659
438243 -350528.806258 -140384.712685
438244 -350669.940311 -131424.675835
...               ...            ...
440875  371319.110964  133166.524410
440876  371482.524421  141320.872736
440877  371655.653689  149475.251732
440878  371838.499941  157629.663144
440879  372031.064418  165784.108715

[2640 rows x 2 columns]
nº de coordenadas unicas: 2640
entries no df inteiro: 18480
18480


In [35]:
(camaras2017[['3763_lat', '3763_long']].iloc[:-2640].values == camaras2017[['3763_lat', '3763_long']].iloc[2640:].values).all()

np.True_

In [36]:
(camaras2018[['3763_lat', '3763_long']].iloc[:-2640].values == camaras2018[['3763_lat', '3763_long']].iloc[2640:].values).all()

np.True_

In [37]:
"""CONCLUSÃO: coordenadas repetem passado 2640 entradas"""

'CONCLUSÃO: coordenadas repetem passado 2640 entradas'

In [38]:
treecam2017 = camaras2017.iloc[:2640]
print(treecam2017.iloc[-1])
coord_array = treecam2017[['3763_lat', '3763_long']].to_numpy()
station_array = gdf[['3763_lat', '3763_long']].to_numpy()
tree = cKDTree(station_array)
distances, indices = tree.query(coord_array, k=1)

time         2017-10-14 11:30:00
lat                         43.0
lon                         -6.1
pm10                   32.025784
3763_long          165784.108715
3763_lat           372031.064418
Name: 757679, dtype: object


In [39]:
treecam2017['nearest_station'] = gdf.iloc[indices]['nome_estac'].values

print(treecam2017["nearest_station"].unique())
print(len(treecam2017["nearest_station"].unique()))

['David Neto' 'Malpique' 'Joaquim Magalhães' 'Cerro' 'Monte Chãos'
 'Sonega' 'Terena' 'Cascais - Escola da Cidadela' 'Santiago do Cacém'
 'Monte Velho' 'Paio Pires' 'Arcos' 'Quebedo' 'Camarinha' 'Fernando Pó'
 'Laranjeiro' 'Quinta do Marquês' 'Fidalguinhos' 'Restelo' 'Beato'
 'Lavradio' 'Mem Martins' 'Odivelas-Ramada' 'Olivais' 'Alverca'
 'Loures-Centro' 'Chamusca' 'Lourinhã' 'Fundão' 'Ervedeira'
 'Instituto Geofísico de Coimbra' 'Montemor-o-Velho'
 'Coimbra/ Avenida Fernão Magalhães' 'Fornelo do Monte' 'Ílhavo' 'Aveiro'
 'Estarreja' 'Santa Combinha' 'Meco-Perafita' 'Anta-Espinho' 'Douro Norte'
 'Sobreiras-Lordelo do Ouro' 'Avintes'
 'Pe Moreira Neves-Castelões de Cepeda' 'Mindelo-Vila do Conde'
 'Seara-Matosinhos' 'Leça do Balio-Matosinhos' 'Ermesinde-Valongo'
 'D.Manuel II-Vermoim' 'Burgães-Santo Tirso' 'Paços de Ferreira'
 'Cónego Dr. Manuel Faria-Azurém' 'Frossos-Braga'
 'Fr Bartolomeu Mártires-S.Vitor' 'Minho-Lima']
55


/tmp/ipykernel_3406/1059474684.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  treecam2017['nearest_station'] = gdf.iloc[indices]['nome_estac'].values


In [40]:
distance_matrix = cdist(coord_array, station_array)
station_idx, coord_idx = linear_sum_assignment(distance_matrix.T)
treecam2017 = treecam2017.copy()
treecam2017['nearest_station'] = None
treecam2017.iloc[coord_idx, treecam2017.columns.get_loc('nearest_station')] = gdf.iloc[station_idx]['nome_estac'].values

print(treecam2017["nearest_station"].unique())
print(len(treecam2017["nearest_station"].unique()))
print(treecam2017["nearest_station"])

[None 'Joaquim Magalhães' 'David Neto' 'Malpique' 'Cerro' 'Sonega'
 'Monte Chãos' 'Santiago do Cacém' 'Monte Velho' 'Quebedo' 'Camarinha'
 'Restelo' 'Laranjeiro' 'Paio Pires' 'Fidalguinhos' 'Arcos' 'Fernando Pó'
 'Terena' 'Cascais - Escola da Cidadela' 'Quinta do Marquês'
 'Alfragide/Amadora' 'Avenida da Liberdade' 'Beato' 'Escavadeira'
 'Lavradio' 'Mem Martins' 'Reboleira' 'Santa Cruz de Benfica'
 'Entrecampos' 'Olivais' 'Odivelas-Ramada' 'Loures-Centro' 'Alverca'
 'Lourinhã' 'Chamusca' 'Ervedeira' 'Montemor-o-Velho'
 'Coimbra/ Avenida Fernão Magalhães' 'Instituto Geofísico de Coimbra'
 'Fundão' 'Ílhavo' 'Aveiro' 'Fornelo do Monte' 'Estarreja' 'Anta-Espinho'
 'João Gomes Laranjo-S.Hora' 'Sobreiras-Lordelo do Ouro'
 'Francisco Sá Carneiro-Campanha' 'Avintes' 'Meco-Perafita'
 'Seara-Matosinhos' 'Custóias-Matosinhos' 'Ermesinde-Valongo'
 'Pe Moreira Neves-Castelões de Cepeda' 'Mindelo-Vila do Conde'
 'VNTelha-Maia' 'D.Manuel II-Vermoim' 'Leça do Balio-Matosinhos'
 'Paços de Ferreira' 'Bu

In [41]:
print(treecam2017["nearest_station"])

755040    None
755041    None
755042    None
755043    None
755044    None
          ... 
757675    None
757676    None
757677    None
757678    None
757679    None
Name: nearest_station, Length: 2640, dtype: object


In [42]:
treecam2017.to_excel("cams/2017_nearest_hungarian.xlsx")

In [43]:
com5_2017 = pd.read_excel('apa/F5media2017.xlsx', header=0)
com19_2017= pd.read_excel('apa/Fmedia2017.xlsx', header=0)

print(com5_2017.head())

        Data    Alverca  Anta-Espinho      Arcos     Aveiro  \
0 2017-10-14  54.300000     42.208333  54.062500  44.791667   
1 2017-10-15  59.066667     63.333333  49.929167  73.458333   
2 2017-10-16  64.258333     54.500000  64.600000  93.000000   
3 2017-10-17  28.808333     17.541667  31.158333  45.000000   
4 2017-10-18  11.125000      9.458333  10.304167  16.083333   

   Avenida da Liberdade    Avintes  Burgães-Santo Tirso  \
0             61.962500  31.500000             4.222222   
1             57.408333  32.000000             1.545455   
2             67.820833        NaN            30.600000   
3             23.591667  22.571429            23.812500   
4             13.220833   9.454545             6.565217   

   Cascais - Escola da Cidadela      Cerro  ...  Santa Cruz de Benfica  \
0                     53.904167  32.041667  ...              71.808333   
1                     57.825000  30.841667  ...              61.808696   
2                           NaN  49.977778  

In [44]:
if com5_2017.shape != com19_2017.shape or not all(com5_2017.columns == com19_2017.columns):
    raise ValueError("DataFrames must have the same shape and column structure")

In [45]:
with open("2017_comparacao.txt", "w") as f:
    for col in com5_2017.columns:
        non_null_df1 = com5_2017[col].notnull().sum()
        non_null_df2 = com19_2017[col].notnull().sum()
        f.write(f"Column: {col}\n")
        f.write(f"  5 mínimos: {non_null_df1}\n")
        f.write(f"  19 mínimos: {non_null_df2}\n")
        f.write("\n")  # Add space between columns

In [46]:
com5_2018 = pd.read_excel('apa/F5media2018.xlsx', header=0)
com19_2018= pd.read_excel('apa/Fmedia2018.xlsx', header=0)

if com5_2018.shape != com19_2018.shape or not all(com5_2018.columns == com19_2018.columns):
    raise ValueError("DataFrames must have the same shape and column structure")

with open("2018_comparacao.txt", "w") as f:
    for col in com5_2018.columns:
        non_null_df1 = com5_2018[col].notnull().sum()
        non_null_df2 = com19_2018[col].notnull().sum()
        f.write(f"Column: {col}\n")
        f.write(f"  5 mínimos: {non_null_df1}\n")
        f.write(f"  19 mínimos: {non_null_df2}\n")
        f.write("\n")  # Add space between columns
        

In [47]:
info_estac = pd.read_excel("apa/estacoes-qualar.xlsx", header=0)
print(info_estac.head())

   Estacao id                    Estacao nome Concelho nome Freguesia nome  \
0        1021             Custóias-Matosinhos    Matosinhos       Custoias   
1        1023               Ermesinde-Valongo       Valongo      Ermesinde   
2        1024             D.Manuel II-Vermoim          Maia        Vermoim   
3        1025                   Meco-Perafita    Matosinhos       Perafita   
4        1028  Francisco Sá Carneiro-Campanha         Porto       Campanhã   

        Zona nome Influencia nome Ambiente nome  Longitude   Latitude  \
0  Porto Litoral            Fundo     Suburbana  -8.644844  41.199806   
1  Porto Litoral            Fundo        Urbana  -8.552600  41.206604   
2  Porto Litoral          Tráfego        Urbana  -8.618715  41.235560   
3  Porto Litoral       Industrial     Suburbana  -8.713092  41.232346   
4  Porto Litoral          Tráfego        Urbana  -8.590847  41.162792   

                           Rua  Altitude estacao Data inicio  \
0         Rua Nova de São Ge

In [48]:
print(gdf.head())

    codigo_est                      nome_estac  data_inici    data_fim  \
0  STA-PT01053                         Avintes  12/07/2010  0000/00/00   
1  STA-PT01021             Custóias-Matosinhos  01/09/1998  0000/00/00   
2  STA-PT01023               Ermesinde-Valongo  01/10/1998  0000/00/00   
3  STA-PT01024             D.Manuel II-Vermoim  01/11/1999  0000/00/00   
4  STA-PT01028  Francisco Sá Carneiro-Campanha  19/09/2000  0000/00/00   

    latitude  longitude   rede_codig                         rede_nome  \
0  41.098187  -8.554351  NET-PT00015  Rede de Qualidade do Ar do Norte   
1  41.199805  -8.644844  NET-PT00015  Rede de Qualidade do Ar do Norte   
2  41.206605  -8.552600  NET-PT00015  Rede de Qualidade do Ar do Norte   
3  41.235563  -8.618715  NET-PT00015  Rede de Qualidade do Ar do Norte   
4  41.162791  -8.590847  NET-PT00015  Rede de Qualidade do Ar do Norte   

  rede_aqd_o                   geometry     3763_long       3763_lat  
0   regional  POINT (-8.55435 41.09819)

In [49]:
df_merged = gdf.merge(
    info_estac[['Estacao nome', 'Estacao id', 'Influencia nome', 'Ambiente nome']],
    left_on='nome_estac',
    right_on='Estacao nome',
    how='left'
)

df_merged = df_merged.drop(columns=['Estacao nome'])
print(df_merged.head())

gdf.to_excel("apa/estacoes_original.xlsx")
df_merged.to_excel("apa/estacoes_infoextra.xlsx")

    codigo_est                      nome_estac  data_inici    data_fim  \
0  STA-PT01053                         Avintes  12/07/2010  0000/00/00   
1  STA-PT01021             Custóias-Matosinhos  01/09/1998  0000/00/00   
2  STA-PT01023               Ermesinde-Valongo  01/10/1998  0000/00/00   
3  STA-PT01024             D.Manuel II-Vermoim  01/11/1999  0000/00/00   
4  STA-PT01028  Francisco Sá Carneiro-Campanha  19/09/2000  0000/00/00   

    latitude  longitude   rede_codig                         rede_nome  \
0  41.098187  -8.554351  NET-PT00015  Rede de Qualidade do Ar do Norte   
1  41.199805  -8.644844  NET-PT00015  Rede de Qualidade do Ar do Norte   
2  41.206605  -8.552600  NET-PT00015  Rede de Qualidade do Ar do Norte   
3  41.235563  -8.618715  NET-PT00015  Rede de Qualidade do Ar do Norte   
4  41.162791  -8.590847  NET-PT00015  Rede de Qualidade do Ar do Norte   

  rede_aqd_o                   geometry     3763_long       3763_lat  \
0   regional  POINT (-8.55435 41.09819

In [50]:
print(Fmedia2017.index)
print(Fmedia2017.head())
longtable2017 = Fmedia2017.reset_index().rename(columns={'index': 'Data'})

Index([2017-10-14, 2017-10-15, 2017-10-16, 2017-10-17, 2017-10-18], dtype='object', name='Data')
              Alverca  Anta-Espinho      Arcos     Aveiro  \
Data                                                        
2017-10-14  54.300000     42.208333  54.062500  44.791667   
2017-10-15  59.066667     63.333333  49.929167  73.458333   
2017-10-16  64.258333     54.500000  64.600000  93.000000   
2017-10-17  28.808333     17.541667  31.158333  45.000000   
2017-10-18  11.125000      9.458333  10.304167  16.083333   

            Avenida da Liberdade    Avintes  Burgães-Santo Tirso  \
Data                                                               
2017-10-14             61.962500  31.500000             4.222222   
2017-10-15             57.408333  32.000000             1.545455   
2017-10-16             67.820833        NaN            30.600000   
2017-10-17             23.591667  22.571429            23.812500   
2017-10-18             13.220833   9.454545             6.565217   

In [51]:
print(longtable2017.index)

RangeIndex(start=0, stop=5, step=1)


In [52]:
longtable2017 = pd.melt(longtable2017, id_vars=['Data'], var_name='Estacao', value_name='pm10')
print(longtable2017)

           Data    Estacao       pm10
0    2017-10-14    Alverca  54.300000
1    2017-10-15    Alverca  59.066667
2    2017-10-16    Alverca  64.258333
3    2017-10-17    Alverca  28.808333
4    2017-10-18    Alverca  11.125000
..          ...        ...        ...
300  2017-10-14  proporção   0.833333
301  2017-10-15  proporção   0.800000
302  2017-10-16  proporção   0.783333
303  2017-10-17  proporção   0.783333
304  2017-10-18  proporção   0.850000

[305 rows x 3 columns]


In [53]:
#print(df_merged.iloc[:, 12])
mapa = df_merged.set_index('nome_estac')['Estacao id'].to_dict()
print(mapa)

{'Avintes': 1053, 'Custóias-Matosinhos': 1021, 'Ermesinde-Valongo': 1023, 'D.Manuel II-Vermoim': 1024, 'Francisco Sá Carneiro-Campanha': 1028, 'João Gomes Laranjo-S.Hora': 1030, 'VNTelha-Maia': 1031, 'Leça do Balio-Matosinhos': 1034, 'Estarreja': 2004, 'Lavradio': 3055, 'Paio Pires': 3063, 'Beato': 3070, 'Olivais': 3071, 'Entrecampos': 3072, 'Avenida da Liberdade': 3075, 'Alfragide/Amadora': 3082, 'Monte Chãos': 4001, 'Monte Velho': 4002, 'Sonega': 4003, 'Santiago do Cacém': 4004, 'Odivelas-Ramada': 3097, 'Loures-Centro': 3085, 'Fr Bartolomeu Mártires-S.Vitor': 1041, 'Frossos-Braga': 1042, 'Pe Moreira Neves-Castelões de Cepeda': 1043, 'Paços de Ferreira': 1044, 'Cónego Dr. Manuel Faria-Azurém': 1046, 'Douro Norte': 1048, 'Joaquim Magalhães': 5007, 'David Neto': 5011, 'Meco-Perafita': 1025, 'Aveiro': 2017, 'Ílhavo': 2018, 'Ervedeira': 2019, 'Instituto Geofísico de Coimbra': 2016, 'Fundão': 2020, 'Reboleira': 3084, 'Restelo': 3087, 'Arcos': 3093, 'Quebedo': 3094, 'Quinta do Marquês': 309

In [54]:
longtable2017['Estacao'] = longtable2017['Estacao'].map(mapa).fillna(longtable2017['Estacao'])
print(longtable2017)

           Data    Estacao       pm10
0    2017-10-14     3101.0  54.300000
1    2017-10-15     3101.0  59.066667
2    2017-10-16     3101.0  64.258333
3    2017-10-17     3101.0  28.808333
4    2017-10-18     3101.0  11.125000
..          ...        ...        ...
300  2017-10-14  proporção   0.833333
301  2017-10-15  proporção   0.800000
302  2017-10-16  proporção   0.783333
303  2017-10-17  proporção   0.783333
304  2017-10-18  proporção   0.850000

[305 rows x 3 columns]


In [ ]:
longtable2017.to_excel("apa/longatable2017_formato1.xlsx", index=False)

In [59]:
last_five = longtable2017.tail(5)
rest = longtable2017.iloc[:-5]
rest_sorted = rest.sort_values(by='Data')
longtable2017_outro_formato = pd.concat([rest_sorted, last_five], ignore_index=True)
#longtable2017_outro_formato.to_excel("apa/longatable2017_formato2.xlsx", index=False)

In [60]:
print(Fmedia2018.index)
print(Fmedia2018.head())
longtable2018 = Fmedia2018.reset_index().rename(columns={'index': 'Data'})

Index([2018-06-16, 2018-06-17, 2018-06-18, 2018-06-19, 2018-06-20, 2018-06-21,
       2018-06-22],
      dtype='object', name='Data')
              Alverca  Anta-Espinho      Arcos     Aveiro  \
Data                                                        
2018-06-16  17.816667     14.833333  17.733333  17.125000   
2018-06-17  16.033333     24.541667  18.320833  19.500000   
2018-06-18  17.416667     21.058824  28.637500  26.416667   
2018-06-19        NaN      6.791667  32.028571  19.541667   
2018-06-20        NaN     13.681818  36.491667  27.125000   

            Avenida da Liberdade    Avintes  Burgães-Santo Tirso  \
Data                                                               
2018-06-16             29.712500  16.611111             9.714286   
2018-06-17             28.112500  18.619048            11.666667   
2018-06-18             37.541667  18.733333            11.041667   
2018-06-19             45.908333  16.045455            16.000000   
2018-06-20             48.4166

In [62]:
longtable2018 = pd.melt(longtable2018, id_vars=['Data'], var_name='Estacao', value_name='pm10')
print(longtable2018)

           Data    Estacao       pm10
0    2018-06-16    Alverca  17.816667
1    2018-06-17    Alverca  16.033333
2    2018-06-18    Alverca  17.416667
3    2018-06-19    Alverca        NaN
4    2018-06-20    Alverca        NaN
..          ...        ...        ...
422  2018-06-18  proporção   0.850000
423  2018-06-19  proporção   0.850000
424  2018-06-20  proporção   0.866667
425  2018-06-21  proporção   0.833333
426  2018-06-22  proporção   0.850000

[427 rows x 3 columns]


In [63]:
longtable2018['Estacao'] = longtable2018['Estacao'].map(mapa).fillna(longtable2018['Estacao'])
print(longtable2018)

           Data    Estacao       pm10
0    2018-06-16     3101.0  17.816667
1    2018-06-17     3101.0  16.033333
2    2018-06-18     3101.0  17.416667
3    2018-06-19     3101.0        NaN
4    2018-06-20     3101.0        NaN
..          ...        ...        ...
422  2018-06-18  proporção   0.850000
423  2018-06-19  proporção   0.850000
424  2018-06-20  proporção   0.866667
425  2018-06-21  proporção   0.833333
426  2018-06-22  proporção   0.850000

[427 rows x 3 columns]


In [64]:
longtable2018.to_excel("apa/longatable2018_formato1.xlsx", index=False)

In [ ]:
"""9 vizinhos - https://www.google.com/maps/d/u/0/edit?mid=1OosQpm0BzMA6xXo_5GvOsZ7JpMByYFU&ll=39.087048646342055%2C-8.049999999999988&z=7"""
print(camaras2017.head())

                      time   lat   lon       pm10      3763_long  \
755040 2017-10-14 11:30:00  36.5 -10.0  42.453941 -167265.612117   
755041 2017-10-14 11:30:00  36.5  -9.9  42.392410 -158305.172793   
755042 2017-10-14 11:30:00  36.5  -9.8  42.162560 -149344.875659   
755043 2017-10-14 11:30:00  36.5  -9.7  41.917164 -140384.712685   
755044 2017-10-14 11:30:00  36.5  -9.6  42.237843 -131424.675835   

             3763_lat  
755040 -350049.532044  
755041 -350218.603496  
755042 -350378.361097  
755043 -350528.806258  
755044 -350669.940311  


In [77]:
cams2017_cortado = camaras2017.iloc[:2640].copy()

geodata2017cams = gpd.GeoDataFrame(
    cams2017_cortado,
    geometry=gpd.points_from_xy(
        cams2017_cortado['3763_long'], 
        cams2017_cortado['3763_lat']
    ),
    crs="EPSG:3763"
)
print(cams2017_cortado.tail(5))


                      time   lat  lon       pm10      3763_long       3763_lat
757675 2017-10-14 11:30:00  43.0 -6.5  45.348019  133166.524410  371319.110964
757676 2017-10-14 11:30:00  43.0 -6.4  40.641777  141320.872736  371482.524421
757677 2017-10-14 11:30:00  43.0 -6.3  36.728497  149475.251732  371655.653689
757678 2017-10-14 11:30:00  43.0 -6.2  32.001675  157629.663144  371838.499941
757679 2017-10-14 11:30:00  43.0 -6.1  32.025784  165784.108715  372031.064418


In [79]:
print(geodata2017cams)

                      time   lat   lon       pm10      3763_long  \
755040 2017-10-14 11:30:00  36.5 -10.0  42.453941 -167265.612117   
755041 2017-10-14 11:30:00  36.5  -9.9  42.392410 -158305.172793   
755042 2017-10-14 11:30:00  36.5  -9.8  42.162560 -149344.875659   
755043 2017-10-14 11:30:00  36.5  -9.7  41.917164 -140384.712685   
755044 2017-10-14 11:30:00  36.5  -9.6  42.237843 -131424.675835   
...                    ...   ...   ...        ...            ...   
757675 2017-10-14 11:30:00  43.0  -6.5  45.348019  133166.524410   
757676 2017-10-14 11:30:00  43.0  -6.4  40.641777  141320.872736   
757677 2017-10-14 11:30:00  43.0  -6.3  36.728497  149475.251732   
757678 2017-10-14 11:30:00  43.0  -6.2  32.001675  157629.663144   
757679 2017-10-14 11:30:00  43.0  -6.1  32.025784  165784.108715   

             3763_lat                         geometry  
755040 -350049.532044  POINT (-167265.612 -350049.532)  
755041 -350218.603496  POINT (-158305.173 -350218.603)  
755042 -3503

In [81]:
geodataestacoes= gpd.GeoDataFrame(
    df_merged,
    geometry=gpd.points_from_xy(
        df_merged['3763_long'], 
        df_merged['3763_lat']
    ),
    crs="EPSG:3763"
)
print(geodataestacoes.iloc[:, -7:])

   rede_aqd_o                         geometry      3763_long       3763_lat  \
0    regional    POINT (-35388.727 158867.677)  -35388.726933  158867.677319   
1    regional     POINT (-42924.77 170193.796)  -42924.770151  170193.796239   
2    regional    POINT (-35183.612 170907.574)  -35183.611562  170907.573844   
3    regional    POINT (-40710.864 174152.466)  -40710.864377  174152.466336   
4    regional    POINT (-38417.072 166057.858)  -38417.071842  166057.858413   
..        ...                              ...            ...            ...   
60   regional    POINT (-41240.984 147938.005)  -41240.983793  147938.005060   
61   regional    POINT (-45229.611 168888.186)  -45229.611077  168888.185576   
62   regional  POINT (-112752.967 -106543.046) -112752.967323 -106543.045842   
63   regional    POINT (103912.518 212147.539)  103912.517850  212147.538525   
64   regional   POINT (-81372.684 -111421.344)  -81372.683667 -111421.343717   

    Estacao id Influencia nome Ambiente

In [86]:
station_coords = np.array(list(geodataestacoes.geometry.apply(lambda geom: (geom.x, geom.y))))
cam_coords = np.array(list(geodata2017cams.geometry.apply(lambda geom: (geom.x, geom.y))))
print(station_coords[:5])
print(cam_coords[:5])

[[-35388.72693303 158867.67731898]
 [-42924.77015087 170193.79623906]
 [-35183.61156175 170907.57384441]
 [-40710.86437655 174152.4663364 ]
 [-38417.07184248 166057.85841318]]
[[-167265.61211736 -350049.53204386]
 [-158305.17279284 -350218.60349628]
 [-149344.87565942 -350378.36109687]
 [-140384.71268495 -350528.80625844]
 [-131424.67583459 -350669.94031139]]


In [90]:
tree = cKDTree(cam_coords)
distances, indices = tree.query(station_coords, k=9)
print(distances)

[[ 3840.27682592  4570.47468883 11560.01294454 11823.0222478
  11938.93013218 12193.00283581 12237.75046911 12968.57787167
  16396.77914271]
 [ 3761.67623677  4626.67855144 11706.06538606 11745.43536609
  12012.49555501 12049.94800892 12149.80591733 13014.96843242
  16453.01209959]
 [ 4042.90126657  4472.50904248 11107.29103369 11270.48607049
  12385.34485523 12490.11046165 12635.88929007 12820.63110169
  16131.20445376]
 [ 4250.02767155  7326.2671149   7878.1262606   9879.7570067
  10710.17440987 12254.45951132 15137.23985821 15707.26084432
  16528.97378594]
 [ 4203.16952641  7015.70047242  8670.63608058 10047.72501252
  10335.40403245 11516.81971632 15257.80215965 16537.64313286
  17035.98080547]
 [ 3468.5922866   5510.76111617 10044.85123309 10920.7819824
  11595.27383193 13032.77584365 13717.18821719 13764.39867905
  14950.09674468]
 [ 6246.60531415  6687.30822309  7331.07521135  7711.49115655
  12833.38852711 13060.92365554 14452.19243608 14656.90871229
  16733.79879287]
 [ 3366.8

In [91]:
print(indices)

[[1854 1855 1814 1815 1894 1895 1853 1856 1813]
 [1894 1893 1854 1934 1853 1933 1895 1892 1855]
 [1894 1895 1934 1935 1893 1854 1855 1896 1933]
 [1894 1934 1893 1933 1895 1935 1854 1892 1853]
 [1894 1854 1895 1893 1855 1853 1934 1896 1935]
 [1893 1894 1853 1854 1892 1933 1934 1895 1852]
 [1933 1893 1934 1894 1932 1892 1935 1895 1973]
 [1894 1893 1934 1933 1895 1854 1892 1853 1935]
 [1734 1694 1735 1695 1733 1693 1736 1696 1774]
 [ 890  889  850  849  891  888  851  930  848]
 [ 849  850  889  848  890  888  809  810  851]
 [ 889  929  888  928  890  930  849  887  848]
 [ 929  889  928  930  888  890  969  968  927]
 [ 889  888  929  928  890  930  887  927  849]
 [ 889  888  929  928  890  887  849  848  930]
 [ 888  928  887  889  927  929  848  886  847]
 [ 612  572  611  571  613  573  610  570  652]
 [ 652  612  653  651  613  611  692  693  691]
 [ 573  572  533  532  574  534  613  612  571]
 [ 613  614  653  612  654  652  573  574  572]
 [ 928  929  927  968  888  969  889  96

In [111]:
weighted_pm10 = []
for i in range(len(geodataestacoes)):
    dists = distances[i]
    idxs = indices[i]

    dists = np.where(dists == 0, 0.001, dists)
    weights = 1 / dists
    pm10_vals = geodata2017cams.iloc[idxs]['pm10'].values

    weighted_avg = np.average(pm10_vals, weights=weights)
    weighted_pm10.append({
        'date': geodata2017cams.iloc[0]['time'], 
        'Estacao id': geodataestacoes.iloc[i]['Estacao id'],
        'Estacao': geodataestacoes.iloc[i]['nome_estac'],
        'pm10_weighted': weighted_avg
    })
df = pd.DataFrame(weighted_pm10)
print(df)

                  date  Estacao id                         Estacao  \
0  2017-10-14 11:30:00        1053                         Avintes   
1  2017-10-14 11:30:00        1021             Custóias-Matosinhos   
2  2017-10-14 11:30:00        1023               Ermesinde-Valongo   
3  2017-10-14 11:30:00        1024             D.Manuel II-Vermoim   
4  2017-10-14 11:30:00        1028  Francisco Sá Carneiro-Campanha   
..                 ...         ...                             ...   
60 2017-10-14 11:30:00        1054                    Anta-Espinho   
61 2017-10-14 11:30:00        1055                Seara-Matosinhos   
62 2017-10-14 11:30:00        3104    Cascais - Escola da Cidadela   
63 2017-10-14 11:30:00        1100                  Santa Combinha   
64 2017-10-14 11:30:00        3095                     Escavadeira   

    pm10_weighted  
0       33.386028  
1       33.233203  
2       32.673066  
3       33.116599  
4       32.977517  
..            ...  
60      35.309360  